In [1]:
!python --version

Python 3.13.2


In [2]:
import re
import os
import zipfile
import gc

# import modin.pandas as pd
import pandas as pd
import numpy as np

# import modin.config as modin_config
# modin_config.Engine.put("dask")
# modin_config.StorageFormat.put("pandas")

In [3]:
# Archive of World Flora Online (WFO) - https://zenodo.org/records/14538251
# families_dwc.tar.gz file contains zipped archives of classification data for all families
# each family is contained within a separate .zip file and the core data are in a file called `classification.csv'
# harvest the following columns from each of  these csv files
# family, genus, specificEpithet & majorGroup

In [42]:
wfo_noduplicates = wfo.drop_duplicates()
wfo_noduplicates.shape

(1149343, 4)

# ___GRooT___

In [5]:
groot = pd.read_csv(r"./GRooTFullVersion.csv", low_memory=False, encoding="latin")

In [6]:
groot_taxonomy = groot.loc[:, ["family", "genus", "species"]].drop_duplicates().reset_index(drop=True)
groot_taxonomy.isna().sum()

family     6135
genus         0
species     138
dtype: int64

In [7]:
# 1959 unique genera in GRoot
groot_taxonomy.genus.drop_duplicates().size

1959

In [10]:
groot_taxonomy.genus.str.title().str.contains("Anona").sum()

np.int64(1)

In [12]:
groot_taxonomy.genus[groot_taxonomy.genus.str.startswith("Mat")]

766       Matteuccia
2913      Matteuccia
3544       Matthiola
4061      Matricaria
4062      Matteuccia
4555      Matricaria
4911       Matthiola
4912       Matthiola
5296      Matricaria
5436         Matayba
5496    Matthiolaria
5521       Matthiola
Name: genus, dtype: object

In [14]:
# looks like GRoot has some spelling mistakes in the genus names!!! 
# YIKES

groot_spelling_corrections = {
"Anona" : "Annona",
"Braciaria" : "Brachiaria",
"Cyanodon" : "Cynodon",
"Helychrisum" : "Helichrysum",
"Matthiolaria" : "Matthiola", # https://list-ui-wfo-staging.rbge.info/taxon/wfo-0000368988-2018-07?page=1 - synonyms
"Rhipogonum" : "Ripogonum",
"Rynchosia" : "Rhynchosia"
}

In [15]:
spelling_corrected_groot_genera = groot_taxonomy.genus.\
        apply(func= lambda g: groot_spelling_corrections.get(g) if g in groot_spelling_corrections.keys() else g.strip().title())

In [16]:
spelling_corrected_groot_genera[spelling_corrected_groot_genera.str.startswith("Ann")]

1167    Annona
2444    Annona
Name: genus, dtype: object

In [19]:
spelling_corrected_groot_genera #.reset_index(drop=True)

0          Betula
1           Picea
2           Pinus
3       Agropyron
4       Artemisia
          ...    
7229    Valeriana
7230     Veronica
7231        Vicia
7232        Vicia
7233    Centaurea
Name: genus, Length: 7234, dtype: object

In [20]:
groot_taxonomy

,family,genus,species
0,NaN,Betula,NaN
1,NaN,Picea,NaN
2,NaN,Pinus,NaN
3,NaN,Agropyron,cristatum
4,NaN,Artemisia,tridentata
...,...,...,...
7229,NaN,Valeriana,excelsa
7230,NaN,Veronica,aphylla
7231,NaN,Vicia,onobrychioides
7232,NaN,Vicia,oroboides


In [23]:
# mhmmmm
groot_taxonomy.genus = spelling_corrected_groot_genera
groot_taxonomy.reset_index(inplace=True, drop=True)

In [24]:
groot_taxonomy

,family,genus,species
0,NaN,Betula,NaN
1,NaN,Picea,NaN
2,NaN,Pinus,NaN
3,NaN,Agropyron,cristatum
4,NaN,Artemisia,tridentata
...,...,...,...
7229,NaN,Valeriana,excelsa
7230,NaN,Veronica,aphylla
7231,NaN,Vicia,onobrychioides
7232,NaN,Vicia,oroboides


In [25]:
# groot_taxonomy.dtypes
groot_taxonomy.family.dtype

dtype('O')

In [61]:
# gc.collect()